In [44]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# define path to SVHN data in your drive
path_to_data = "/content/gdrive/MyDrive/data/SVHN"

Mounted at /content/gdrive


# Setup Notebook

In [1]:
# import standard libraries
import numpy as np

import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, ConcatDataset

import matplotlib.pyplot as plt

In [2]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

Using device cpu


In [3]:
# install weight and biases library
!pip install wandb
import wandb

     |████████████████████████████████| 1.8MB 5.1MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
     |████████████████████████████████| 133kB 36.3MB/s 
     |████████████████████████████████| 174kB 30.7MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=b886989d083dfb9097f07d4cd0ff60c8213f548785620c5a0b178ee1ae4c8a3d
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=ce85925cef13ccc5d143840db23306084502076043d82a8d8104779cdad99231
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [4]:
# setup repo
!rm -r dl-computer-vision
!git clone https://github.com/simonamtoft/dl-computer-vision.git

import sys
sys.path.append('dl-computer-vision')

import models as m
import helpers as h
import data as d

rm: cannot remove 'dl-computer-vision': No such file or directory
Cloning into 'dl-computer-vision'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 191 (delta 89), reused 144 (delta 48), pack-reused 0
Receiving objects: 100% (191/191), 28.23 KiB | 4.03 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [5]:
# Define name of project on weight and biases site
project_name = "project-1-2"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Project 1.2: Object Detection

In [6]:
# login to weight and biases
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
config = {
    'epochs': 20,
    'batch_size': 64,
    'learning_rate': 1e-3,
    'optimizer': 'adam',
    'conv_dim': [
        [16, 5],
        [16, 5], 
        [16, 3],
        [16, 3],
        [32, 3],
        [32, 1],
    ],
    'maxpool_idx': [2, 3],
    'batch_norm': False,
    'step_lr': [True, 1, 0.8],
}

## Load Data

In [26]:
# Define data transforms
svhn_train_transform = transforms.Compose([
  transforms.ToTensor()
])
svhn_test_transform = transforms.Compose([
  transforms.ToTensor()
])

In [27]:
# Download SVHN datasets 
svhn_trainset = datasets.SVHN(
    "./data", 
    split="train", 
    transform=svhn_train_transform, 
    download=True
)
svhn_testset = datasets.SVHN(
    "./data", 
    split="test", 
    transform=svhn_test_transform, 
    download=True
)

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


In [34]:
# get 'corners' data
corner_trainset = d.SVHNCorners(
    train=True, 
    transform=transforms.ToTensor(), 
    data_path=path_to_data
)
corner_trainset, _ = torch.utils.data.random_split(
    corner_trainset, 
    [len(corner_trainset) // 10, len(corner_trainset) - len(corner_trainset)// 10]
)

corner_testset = d.SVHNCorners(
    train=False, 
    transform=transforms.ToTensor(), 
    data_path=path_to_data
)
corner_testset, _ = torch.utils.data.random_split(
    corner_testset, 
    [len(corner_testset) // 10, len(corner_testset) - len(corner_testset)// 10]
)

In [35]:
# Concat datasets and setup loaders
trainset = ConcatDataset([svhn_trainset, corner_trainset])
testset = ConcatDataset([svhn_testset, corner_testset])

# get size
n_train = len(trainset)
n_test = len(testset)

In [30]:
# Get data loaders
train_loader = DataLoader(
    trainset, batch_size=config["batch_size"], shuffle=True, num_workers=2
)
test_loader = DataLoader(
    testset, batch_size=config["batch_size"], shuffle=False, num_workers=2
)

## Train Model

In [ ]:
# Instantiate model
model = m.PureCNN(config).to(device)

# train model
m.train(model, config, project_name, train_loader, test_loader, n_train, n_test)

epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Start of training




epoch:  10%|█         | 2/20 [02:40<24:00, 80.01s/it]

KeyboardInterrupt: ignored